<a href="https://colab.research.google.com/github/uakarsh/TiLT-Implementation/blob/main/how_did_i_prepare_the_stuffs/tilt_part_2_3_sample_preparing_funsd_for_t5_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/uakarsh/TiLT-Implementation.git

Cloning into 'TiLT-Implementation'...
remote: Enumerating objects: 77, done.
remote: Counting objects: 100% (77/77), done.
remote: Compressing objects: 100% (58/58), done.
remote: Total 77 (delta 31), reused 45 (delta 11), pack-reused 0
Unpacking objects: 100% (77/77), 2.77 MiB | 7.51 MiB/s, done.


In [ ]:
!pip install -r /content/TiLT-Implementation/requirements.txt

In [ ]:
import sys
sys.path.append("/content/TiLT-Implementation/src/")

In [ ]:
from transformers import AutoTokenizer
from datasets import load_dataset
import torch
import torch.nn

model_name = "t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast = True)

/usr/local/lib/python3.9/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [ ]:
hf_ds = load_dataset("nielsr/funsd-layoutlmv3")

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset funsd-layoutlmv3 downloaded and prepared to /root/.cache/huggingface/datasets/nielsr___funsd-layoutlmv3/funsd/1.0.0/0e3f4efdfd59aa1c3b4952c517894f7b1fc4d75c12ef01bcc8626a69e41c1bb9. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
from dataset import FUNSDDs
from torchvision import transforms
from tqdm.auto import tqdm

transform = transforms.Compose([transforms.ToTensor(), 
                                transforms.Lambda(lambda x : 2 * x - 1)])

In [ ]:
def get_id2label_and_label2id():
    label2id = {'O': 0, 'B-HEADER': 1, 'I-HEADER': 2, 'B-QUESTION': 3, 'I-QUESTION': 4, 'B-ANSWER': 5, 'I-ANSWER': 6}
    id2label = {0: 'O', 1: 'B-HEADER', 2: 'I-HEADER', 3: 'B-QUESTION', 4: 'I-QUESTION', 5: 'B-ANSWER', 6: 'I-ANSWER'}
    return id2label, label2id

id2label, label2id = get_id2label_and_label2id()

def convert_id_to_label(list_of_label):
  return [id2label[x] for x in list_of_label]

In [ ]:
train_new_tags = list(map(lambda x : convert_id_to_label(x), hf_ds['train']['ner_tags']))
test_new_tags = list(map(lambda x : convert_id_to_label(x), hf_ds['test']['ner_tags']))

In [ ]:
hf_ds['train'] = hf_ds['train'].remove_columns("ner_tags").add_column("ner_tags", train_new_tags)
hf_ds['test'] = hf_ds['test'].remove_columns("ner_tags").add_column("ner_tags", test_new_tags)

In [ ]:
train_ds = FUNSDDs(hf_ds['train'],tokenizer = tokenizer, transform = transform)
val_ds = FUNSDDs(hf_ds['test'],tokenizer = tokenizer, transform = transform)

In [ ]:
class CollateFn(object):
  def __init__(self, tokenizer):
    self.tokenizer = tokenizer

  def __call__(self, list_of_ds):
    simple_keys = ["input_ids", "attention_mask", "bboxes", "pixel_values" ]
    actual_batch = {}
    for key in simple_keys:
      actual_batch[key] = torch.stack([x[key] for x in list_of_ds])
    
    actual_batch['labels'] = self.tokenizer.batch_encode_plus([x['labels'] for x in list_of_ds], return_tensors = 'pt', is_split_into_words = True,
                                                              padding='max_length', truncation = True)['input_ids']
    return actual_batch


In [ ]:
collate_fn = CollateFn(tokenizer)

In [ ]:
sample_batch_encoding = collate_fn([train_ds[0], train_ds[1]])

In [ ]:
for key in sample_batch_encoding:
  print(f"Key : {key}, has shape : {sample_batch_encoding[key].shape}")

Key : input_ids, has shape : torch.Size([2, 512])
Key : attention_mask, has shape : torch.Size([2, 512])
Key : bboxes, has shape : torch.Size([2, 512, 4])
Key : pixel_values, has shape : torch.Size([2, 3, 384, 512])
Key : labels, has shape : torch.Size([2, 512])
